# Fase 1


#### check y limpieza
#### link to dataset: https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs?resource=download

### Cargo Dataset Kaggle 

In [25]:
import pandas as pd


# Cargo el csv
file_path = r"C:\Users\edo\Desktop\Final Project\Kaggle - Billboard Hot 100 dataset.csv"  

df = pd.read_csv(file_path)

df.head()


,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


In [27]:
df.isna().sum()

date                  0
rank                  0
song                  0
artist                0
last-week         32312
peak-rank             0
weeks-on-board        0
dtype: int64

### Conversiones Basicas

- Converson 'date' a datetime y creacion columna 'year'

In [28]:
#  Converson 'date' a datetime y creacion columna 'year'
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# creacion columna de busqueda combinando cancion y artista
df['song'] = df['song'].str.strip() 
df['artist'] = df['artist'].str.strip()
df['song_artist'] = df['song'] + " - " + df['artist']

df.tail()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,song_artist
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1,1958,Over And Over - Thurston Harris
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1,1958,I Believe In You - Robert & Johnny
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1,1958,Little Serenade - The Ames Brothers
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1,1958,I'll Get By (As Long As I Have You) - Billy Wi...
330086,1958-08-04,100,Judy,Frankie Vaughan,NaN,100,1,1958,Judy - Frankie Vaughan


### Check duplicados y consistencia

In [ ]:
cols = [c for c in df.columns if df[c].apply(lambda x: isinstance(x,list)).sum() == 0]

# Duplicados absoludos
total_duplicates_absolute = df[cols].duplicated().sum()
print(f"Duplicados absolutos: {total_duplicates_absolute}")



Duplicados absolutos: 0


In [ ]:
# check duplicados por track_id
dup_track = df.duplicated(subset=["song"], keep=False)

print("Tracks duplicados (esperado: multiplas semanas en lista):", dup_track.sum())
print("Tracks unicos:", df['song'].nunique())
print("Filas totales:", len(df))


Tracks duplicados (esperado: multiplas semanas en lista): 328194
Tracks unicos: 24619
Filas totales: 330087


### guardo el dataset COMPLETO

In [31]:
df.to_csv(r"C:\Users\edo\Desktop\Final Project\billboard_weekly_ready.csv", index=False)

print("Guardado")


Guardado


### OPTIONAL Metodo de calculo Top 100 annual

**avg_rank**  
Media de la posicion semanal de un brano por anyo:

avg_rank = sum(rank weekly) / num  'weeks on board' 

- Mas bajo el valor, mas alta la posicion en el anyo
- las otras metricas (peak_rank, weeks_on_board) quedan en el dataset, aunque no influyen en la clasifica annual


In [32]:
# Agrupar por ano y cancion/artista
# 
# avg_rank         -->  promedio de posiciones semanales
# peak_rank        -->  mejor posicion alcanzada
# weeks_on_board   -->  total semanas en la lista
annual_dataset = df.groupby(['year', 'song', 'artist', 'song_artist'], as_index=False).agg(
    avg_rank=('rank', 'mean'), 
    peak_rank=('peak-rank', 'min'),
    weeks_on_board=('weeks-on-board', 'sum'))



# Ordenar por anyo y avg_rank ascendente (el mas popular primero)
# avg_rank y peak_rank ascendente, weeks_on_board descendente
# en caso de empate de avg_rank, se mira peak_rank y luego weeks_on_board
annual_dataset = annual_dataset.sort_values(
    ['year', 'avg_rank', 'peak_rank', 'weeks_on_board'],
    ascending=[True, True, True, False] 
)

# Seleccionar top 100 por ano
top100_annual = annual_dataset.groupby('year').head(100).copy()
# avr_rank round 2 --> 2 decimales
top100_annual['avg_rank'] = top100_annual['avg_rank'].round(2)

# Agregar columna de posicion dentro del ano usando cumcount()
# cumcount(): cuenta filas acumuladas por grupo, comenzando en 0
top100_annual['position'] = top100_annual.groupby('year').cumcount() + 1



In [33]:

# Crear columna con todos los artistas originales antes de limpiar
top100_annual['artist_full'] = top100_annual['artist']


# Función para limpiar el nombre del artista principal
def clean_artist(artist):
    # toma solo el artista principal antes de Featuring/Feat/ft
    artist = re.split('Featuring|Feat|ft', artist, flags=re.IGNORECASE)[0].strip()
    # reemplaza & con 'and'
    artist = artist.replace('&', 'and')
    return artist

# Aplicar limpieza a columna artist
top100_annual['artist_main'] = top100_annual['artist'].apply(clean_artist)

#  Crear columna song_and_artist lista para consultas Spotify
top100_annual['song_and_artist'] = top100_annual['song'] + " - " + top100_annual['artist_main']

#top100_annual.head()

In [34]:
# Seleccionar columnas finales 
top100_annual = top100_annual[['year', 'position', 'song', 'artist_main', 'artist_full', 'song_and_artist','avg_rank', 'peak_rank', 'weeks_on_board']]

top100_annual[top100_annual['year'] == 1964].head(15)

,year,position,song,artist_main,artist_full,song_and_artist,avg_rank,peak_rank,weeks_on_board
4296,1964,1,I Feel Fine,The Beatles,The Beatles,I Feel Fine - The Beatles,7.50,1,10
4321,1964,2,I Want To Hold Your Hand,The Beatles,The Beatles,I Want To Hold Your Hand - The Beatles,7.53,1,120
4702,1964,3,There! I've Said It Again,Bobby Vinton,Bobby Vinton,There! I've Said It Again - Bobby Vinton,9.88,1,76
4432,1964,4,Louie Louie,The Kingsmen,The Kingsmen,Louie Louie - The Kingsmen,9.88,2,100
4114,1964,5,Can't Buy Me Love,The Beatles,The Beatles,Can't Buy Me Love - The Beatles,11.30,1,55
4165,1964,6,Dominique,The Singing Nun (Soeur Sourire),The Singing Nun (Soeur Sourire),Dominique - The Singing Nun (Soeur Sourire),11.60,1,55
4567,1964,7,She Loves You,The Beatles,The Beatles,She Loves You - The Beatles,12.00,1,120
4018,1964,8,A Hard Day's Night,The Beatles,The Beatles,A Hard Day's Night - The Beatles,12.00,1,91
4073,1964,9,Baby Love,The Supremes,The Supremes,Baby Love - The Supremes,12.46,1,91
4629,1964,10,Surfin' Bird,The Trashmen,The Trashmen,Surfin' Bird - The Trashmen,12.89,4,81


In [35]:
top_per_year = top100_annual[top100_annual['position'] == 1]
top_per_year.tail(10)


,year,position,song,artist_main,artist_full,song_and_artist,avg_rank,peak_rank,weeks_on_board
30315,2012,1,Die Young,Ke$ha,Ke$ha,Die Young - Ke$ha,5.67,2,78
31117,2013,1,The Monster,Eminem,Eminem Featuring Rihanna,The Monster - Eminem,1.86,1,28
31530,2014,1,Shake It Off,Taylor Swi,Taylor Swift,Shake It Off - Taylor Swi,2.82,1,153
31861,2015,1,Hello,Adele,Adele,Hello - Adele,1.00,1,28
32264,2016,1,Closer,The Chainsmokers,The Chainsmokers Featuring Halsey,Closer - The Chainsmokers,2.10,1,210
33106,2017,1,Rockstar,Post Malone,Post Malone Featuring 21 Savage,Rockstar - Post Malone,1.38,1,91
33861,2018,1,"Thank U, Next",Ariana Grande,Ariana Grande,"Thank U, Next - Ariana Grande",1.14,1,28
34081,2019,1,Circles,Post Malone,Post Malone,Circles - Post Malone,4.06,1,136
35373,2020,1,Willow,Taylor Swi,Taylor Swift,Willow - Taylor Swi,1.00,1,1
35935,2021,1,Stay,The Kid LAROI and Justin Bieber,The Kid LAROI & Justin Bieber,Stay - The Kid LAROI and Justin Bieber,2.12,1,136


In [36]:
# Top 100 anual generada basada en avg_rank
# Guardar CSV
top100_annual.to_csv(r"C:\Users\edo\Desktop\Final Project\billboard_top100_annual.csv", index=False)


print("billboard_top100_annual -->  Guardado")


billboard_top100_annual -->  Guardado
